# TwitterAPI - Get tweets

In [1]:
import tweepy
import json
import pandas as pd
import csv
import time
import os
from dotenv import load_dotenv
load_dotenv()

True

## Keys and tokens for the project

In [2]:
consumer_key = os.getenv('consumer_key')
consumer_secret = os.getenv('consumer_secret')
access_token = os.getenv('access_token')
access_token_secret = os.getenv('access_token_secret')

## Autenticacion

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

## Get Tweets

In [16]:
def getTweets(query_string, numPages, numTweets, geoCode):
  # We cannot make large API call in one go.
  # Define a pandas dataframe to store the date:
  db_tweets = pd.DataFrame(columns = ['name', 'username', 'acctdesc', 'user_location', 'tweet_created_at',
   'tweet_full_text', 'tweet_place_name', 'tweet_place_type', 'tweet_place_bounding_box_type', 'tweet_place_bounding_box_coordinates', 'tweet_url'])
  program_start = time.time()
  i = 0
  while(numPages != i):
    i+=1
    try:
      pages = tweepy.Cursor(api.search, q=query_string, tweet_mode='extended', count=numTweets).pages(i)
      # Begin scraping the tweets:
      for page in pages:
        print('Tweets obtenidos de la página {}: {}'.format(i,len(page)))
        for tweet in page:
          start_url = 'https://twitter.com/twitter/statuses/'
          if not tweet.full_text.startswith('RT'):
            # Pull the values
            tweet_id = tweet.id
            name = tweet.user.name
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            user_location = tweet.user.location
            tweet_created_at = tweet.created_at
            tweet_full_text = tweet.full_text
            tweet_place_bounding_box_type = tweet.place.bounding_box.type
            tweet_place_bounding_box_coordinates = tweet.place.bounding_box.coordinates
            tweet_url = start_url + str(tweet_id)
            tweet_place_name =  tweet.place.name
            tweet_place_type = tweet.place.place_type
            # except AttributeError:
            #   tweet_place_name =  'None'
            #   tweet_place_type = 'None'
        # Data for the dataset:
            ith_tweet = [name, username, acctdesc, user_location, tweet_created_at, tweet_full_text, tweet_place_name, tweet_place_type, tweet_place_bounding_box_type, tweet_place_bounding_box_coordinates, tweet_url]
          # Append to db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
    except tweepy.TweepError:
      print('Too much requests.... let\'s take a break')
      time.sleep(60*150)
      continue
    # time.sleep(920) #15 minute sleep time
    # Store dataframe in csv
  path_file = './../datasets/tweets'+geoCode+'.csv'
  db_tweets.to_csv(path_file, index = False, encoding='utf-8')
    
  program_end = time.time()
  print('El proceso se ha completado!')
  print('Total time: {} minutes.'.format(round(program_end - program_start)/60, 2))

In [17]:
# Initialise these variables:
geoCode = 'EC'
places = api.geo_search(query=geoCode, granularity="country")
place_id = places[0].id
query_string = "#PlanVacunación9100 OR #VacunaCOVID19 OR #JuntosLoLogramos place:"+place_id
numTweets = 100
numPages = 50
# Call the function scraptweets
getTweets(query_string, numPages, numTweets, geoCode)

Too much requests.... let's take a break


In [5]:
pd.read_csv('./../datasets/tweetsEC.csv')

,name,username,acctdesc,user_location,tweet_created_at,tweet_full_text,tweet_place_name,tweet_place_type,tweet_place_bounding_box_type,tweet_place_bounding_box_coordinates,tweet_url
0,Ramiro Diaz Poggi,RamiroDiazPogg1,Benjamin la pregunta y respuesta a todo || Dir...,"Portoviejo, Ecuador",2021-08-17 01:02:32,Hoy tuve la oportunidad de acompañar a @juanfr...,Portoviejo,city,Polygon,"[[[-80.5848819972519, -1.20098999966436], [-80...",https://twitter.com/twitter/statuses/142743564...
1,Infocentro Cazaderos,Info_Cazaderos,NaN,"Cazaderos,Zapotillo, Ecuador",2021-08-16 22:42:47,El #InfocentroCazaderos brinda capacitaciones...,Zapotillo,city,Polygon,"[[[-80.4869009961082, -4.48582301946279], [-80...",https://twitter.com/twitter/statuses/142740047...
2,LArry Vite Cevallos,Larry_vite,Alcalde de SANTA ROSA,Santa Rosa - El Oro - Ecuador,2021-08-16 22:28:00,VAMOS CON TODO !! A VACUNARNOS 2da Dosis 🤗🤗 C...,Santa Rosa,city,Polygon,"[[[-80.3055469971365, -3.61982300010924], [-79...",https://twitter.com/twitter/statuses/142739675...
3,Margarita Meier de Caballero.,MeierMargarita,Ex Docente de la Universidad de Guayaquil y Un...,NaN,2021-08-16 22:16:27,@LassoGuillermo @guido_chiriboga #JuntosLoLogr...,Guayaquil,city,Polygon,"[[[-80.4676050033072, -3.06371600046572], [-79...",https://twitter.com/twitter/statuses/142739384...
4,Agricultura Santo Domingo,AgriculturaSDTS,Cuenta Oficial Dirección Distrital de Santo Do...,"Santo Domingo, Ecuador",2021-08-16 22:11:20,Junto a varias autoridades de entidades públic...,Santo Domingo,city,Polygon,"[[[-79.5480669992699, -0.700181999346273], [-7...",https://twitter.com/twitter/statuses/142739255...
...,...,...,...,...,...,...,...,...,...,...,...
6759,edusuperiorec,EduSuperiorEc,Cuenta Oficial de la Secretaría de Educación S...,Ecuador,2021-08-09 00:21:04,¡#DileSíALaVacuna por el Ecuador!\n\nAl comple...,Quito,city,Polygon,"[[[-78.948465999276, -0.588618000469648], [-78...",https://twitter.com/twitter/statuses/142452610...
6760,Gobernación de Loja,GoberLoja,Cuenta oficial de Gobernación de la provincia ...,Loja,2021-08-09 00:11:04,#IntendenciaDePolicíaLoja y #ComisaríaDePolicí...,Loja,city,Polygon,"[[[-79.546383997495, -4.51083499992033], [-79....",https://twitter.com/twitter/statuses/142452358...
6761,edusuperiorec,EduSuperiorEc,Cuenta Oficial de la Secretaría de Educación S...,Ecuador,2021-08-08 23:52:07,Vamos por la segunda dosis. ¡Juntos lograremos...,Quito,city,Polygon,"[[[-78.948465999276, -0.588618000469648], [-78...",https://twitter.com/twitter/statuses/142451881...
6762,MSc Douglas Torres Feraud 🇪🇨,douglastorresfe,Consultor Asesor de Rendicion de Cuentas y Tra...,Guayaquil - Ecuador,2021-08-08 20:36:37,@LassoGuillermo mas vale prevenir q lamentar ....,Guayaquil,city,Polygon,"[[[-80.4676050033072, -3.06371600046572], [-79...",https://twitter.com/twitter/statuses/142446961...
